<a href="https://colab.research.google.com/github/bafanaS/dim-reduction-with-cnn-lstm/blob/main/CNN_or_CNN_LSTM_for_Hand_%26_Tongue_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN / CNN-LSTM for Hand and Tongue

This creates proof of concept for using the CNN / CNN-LSTM architecture for further research

In [1]:
# Necessary Imports

import numpy as np
import pandas as pd

In [2]:
# Settings + Imports

from matplotlib import rcParams
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
# Load the data

import os, requests

fname = 'motor_imagery.npz'
url = "https://osf.io/ksqv8/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)

In [4]:
alldat = np.load(fname, allow_pickle=True)['dat']

In [5]:
participant = 2

In [6]:
# Extract out the real data from a partiuclar participant

from scipy import signal
import numpy as np

# Find resting state intervals and add to tongue/hand interval list.
intervals = [(on, off) for on, off in zip(alldat[participant][0]['t_on'], alldat[participant][0]['t_off'])]
resting = [(intervals[i][1], intervals[i+1][0]) for i in range(0, len(intervals)-1)]
intervals = np.append(np.array(intervals), np.array(resting), axis = 0)

# Create a set of classes and prime for classification
classes = np.append(np.array([0 if i < 12 else 1 for i in alldat[participant][0]['stim_id']]), np.repeat(2, len(resting)))

channels = len(alldat[participant][0]['locs'])
stims = len(intervals)

X = np.empty((stims, 3000, channels))
y = np.empty((stims))

for i in range(stims):

    t_on = intervals[i][0]
    label = classes[i]

    # Real data only

    V = alldat[participant][0]['V'].astype('float32')
    b, a = signal.butter(3, [50], btype='high', fs=1000)
    V = signal.filtfilt(b, a, V, 0)

    V = np.abs(V)**2
    b, a = signal.butter(3, [10], btype='low', fs=1000)
    V = signal.filtfilt(b, a, V, 0)

    V_real = V/V.mean(0)
    V_real = V_real[t_on : t_on + 3000]

    X[i] = V_real
    y[i] = label

# Data Processing

In [7]:
# Train and Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [9]:
X_train = np.expand_dims(X_train, axis=3)

In [10]:
X_test = np.expand_dims(X_test, axis=3)

In [11]:
X_train.shape

(89, 3000, 48, 1)

In [12]:
X_test.shape

(30, 3000, 48, 1)

In [13]:
y_train = to_categorical(y_train)

In [14]:
y_test = to_categorical(y_test)

# Keras Tuner

In [27]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.9 MB/s eta 0:00:00


### CNN-LSTM KerasTuner

In [28]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import losses


def build_model(hp: HyperParameters):
    model = Sequential()

    # CNN layers
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(3000, channels, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=16, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # # LSTM layers
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(units=hp.Int('lstm_units', min_value=50, max_value=200, step=50),
                   activation='tanh'))

    # Dense layers
    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_1_units', min_value=16, max_value=128, step=16),
                    activation='relu'))
    model.add(Dense(units=hp.Int('dense_2_units', min_value=8, max_value=64, step=8),
                    activation='relu'))
    model.add(Dense(3, activation='softmax')) # 2 classes: real or imaginary

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



Using TensorFlow backend


<ipython-input-28-b54a363066e0>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


### CNN KerasTuner



In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import losses


def build_model(hp: HyperParameters):
    model = Sequential()

    # CNN layers
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=(3000, channels, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=16, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Dense layers
    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_1_units', min_value=16, max_value=128, step=16),
                    activation='relu'))
    model.add(Dense(units=hp.Int('dense_2_units', min_value=8, max_value=64, step=8),
                    activation='relu'))
    model.add(Dense(3, activation='softmax')) # 2 classes: real or imaginary

    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='LOG')),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model



<ipython-input-18-2782bb02e27e>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


### Evaluation

In [29]:

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    directory='output',
    project_name='Brain_Activity_Classification'
)



In [30]:
y_train_integers = np.argmax(y_train, axis=1)
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_integers), y=y_train_integers)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Pass class weights to the search method
tuner.search(X_train, y_train, batch_size=25, epochs=50, validation_data=(X_test, y_test), class_weight=class_weights_dict)

best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 4 Complete [00h 01m 25s]
val_accuracy: 0.2666666805744171

Best val_accuracy So Far: 0.5666666626930237
Total elapsed time: 00h 04m 59s

Search: Running Trial #5

Value             |Best Value So Far |Hyperparameter
80                |112               |conv_1_filter
3                 |5                 |conv_1_kernel
64                |32                |conv_2_filter
5                 |3                 |conv_2_kernel
100               |50                |lstm_units
32                |64                |dense_1_units
48                |56                |dense_2_units
0.00054021        |4.3239e-05        |learning_rate



Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7a3a700ec280>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


Epoch 1/50


KeyboardInterrupt: ignored

In [ ]:
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=1)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

#### Saving CNN LSTM

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model.save_weights(f"cnn-lstm-ht-p{participant}.h5")

# Print the best hyperparameters
print("Best Hyperparameters:")
print(f"Conv_1 Filter: {best_hyperparameters.get('conv_1_filter')}")
print(f"Conv_1 Kernel Size: {best_hyperparameters.get('conv_1_kernel')}")
print(f"Conv_2 Filter: {best_hyperparameters.get('conv_2_filter')}")
print(f"Conv_2 Kernel Size: {best_hyperparameters.get('conv_2_kernel')}")
print(f"LSTM Units: {best_hyperparameters.get('lstm_units')}")
print(f"Dense_1 Units: {best_hyperparameters.get('dense_1_units')}")
print(f"Dense_2 Units: {best_hyperparameters.get('dense_2_units')}")
print(f"Learning Rate: {best_hyperparameters.get('learning_rate')}")

#### Saving CNN

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model.save_weights(f"cnn-ht-p{participant}.h5")

# Print the best hyperparameters
print("Best Hyperparameters:")
print(f"Conv_1 Filter: {best_hyperparameters.get('conv_1_filter')}")
print(f"Conv_1 Kernel Size: {best_hyperparameters.get('conv_1_kernel')}")
print(f"Conv_2 Filter: {best_hyperparameters.get('conv_2_filter')}")
print(f"Conv_2 Kernel Size: {best_hyperparameters.get('conv_2_kernel')}")
# print(f"LSTM Units: {best_hyperparameters.get('lstm_units')}")
print(f"Dense_1 Units: {best_hyperparameters.get('dense_1_units')}")
print(f"Dense_2 Units: {best_hyperparameters.get('dense_2_units')}")
print(f"Learning Rate: {best_hyperparameters.get('learning_rate')}")

# Best LSTM-CNN

In [31]:
# All values taken from the best performing CNN LSTM
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

# CNN layers
model.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(3000, channels, 1))) # Adjust channels as per your dataset
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# LSTM layers
model.add(TimeDistributed(Flatten()))
model.add(LSTM(units=150, activation='tanh'))

# Dense layers
model.add(Dense(units=96, activation='relu'))
model.add(Dense(units=24, activation='relu'))
model.add(Dense(3, activation='softmax')) # 3 classes: adjust according to your problem

model.compile(optimizer=Adam(learning_rate=6.261461260984394e-05),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model.load_weights('cnn_lstm-ht-p2.h5')
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")



1/1 [==============================] - 4s 4s/step - loss: 0.8510 - accuracy: 0.7333
Test loss: 0.8509567379951477
Test accuracy: 0.7333333492279053


'\nTest loss: 0.8509567379951477\nTest accuracy: 0.7333333492279053\n'

# CNN BEST VALUES

In [39]:
# For best performing CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Best hyperparameters
conv_1_filter = 64
conv_1_kernel_size = 3
conv_2_filter = 48
conv_2_kernel_size = 5
dense_1_units = 96
dense_2_units = 48
learning_rate = 1.202075262469187e-05

# Defining the model
model = Sequential()

# CNN layers
model.add(Conv2D(filters=conv_1_filter, kernel_size=(conv_1_kernel_size, conv_1_kernel_size),
                 activation='relu', input_shape=(3000, channels, 1))) # Replace 'channels' with the actual value
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=conv_2_filter, kernel_size=(conv_2_kernel_size, conv_2_kernel_size),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dense layers
model.add(Flatten())
model.add(Dense(units=dense_1_units, activation='relu'))
model.add(Dense(units=dense_2_units, activation='relu'))
model.add(Dense(3, activation='softmax')) # 3 classes: resting, hand, tongue

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])




Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 2998, 46, 64)      640       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 1499, 23, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 1495, 19, 48)      76848     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 747, 9, 48)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 322704)            0         
                                                                 
 dense_9 (Dense)             (None, 96)               

In [41]:
model.load_weights('cnn-ht-p2.h5')
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)


print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")



1/1 [==============================] - 0s 83ms/step - loss: 0.7085 - accuracy: 0.8667
Test loss: 0.7084587812423706
Test accuracy: 0.8666666746139526
